In [12]:
import pandas as pd
df = pd.read_excel('C:/Users/px/Downloads/Input.xlsx')

In [2]:
df['URL']

0      https://insights.blackcoffer.com/rise-of-telem...
1      https://insights.blackcoffer.com/rise-of-e-hea...
2      https://insights.blackcoffer.com/rise-of-e-hea...
3      https://insights.blackcoffer.com/rise-of-telem...
4      https://insights.blackcoffer.com/rise-of-telem...
                             ...                        
109    https://insights.blackcoffer.com/coronavirus-i...
110    https://insights.blackcoffer.com/coronavirus-i...
111    https://insights.blackcoffer.com/what-are-the-...
112    https://insights.blackcoffer.com/marketing-dri...
113    https://insights.blackcoffer.com/continued-dem...
Name: URL, Length: 114, dtype: object

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract the heading and paragraphs from an article URL
def extract_article_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        heading = soup.find('h1').text.strip()
        paragraphs = [p.text.strip() for p in soup.find_all('p')]
        content = str(heading) + str(paragraphs)
        return content
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error processing {url}: {str(e)}")
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
    return None

content_list = []

for link in df['URL']:
    content = extract_article_content(link)
    if content:
        content_list.append(content)
    else:
        # If there was an error, append a placeholder or empty string
        content_list.append('')

# Assign the content list to the 'data' column in the DataFrame
df['data'] = content_list

# Now df['data'] will have the extracted content or empty strings for URLs with errors


HTTP Error processing https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
HTTP Error processing https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [22]:
import pandas as pd
from textblob import TextBlob
import nltk
nltk.download('punkt')
from textstat import flesch_kincaid_grade

def calculate_scores(text):
    if not text:
        # If the text is empty, return 0 for all scores
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    # Calculate sentiment scores
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    
    # Calculate complexity and FOG index
    words = nltk.word_tokenize(text)
    num_words = len(words)
    num_complex_words = sum(1 for word in words if len(word) > 6)
    complex_word_percentage = (num_complex_words / num_words) * 100
    fog_index = flesch_kincaid_grade(text)
    
    # Calculate average word length and syllables per word
    word_lengths = [len(word) for word in words]
    avg_word_length = sum(word_lengths) / len(word_lengths)
    total_syllables = sum(sum(1 for _ in nltk.re.findall(r'[aeiouyAEIOUY]+', word)) for word in words)
    syllables_per_word = total_syllables / num_words
    
    # Count personal pronouns
    personal_pronouns = ["I", "me", "my", "mine", "myself", "you", "your", "yours", "yourself", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "we", "us", "our", "ours", "ourselves", "they", "them", "their", "theirs", "themselves"]
    personal_pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)
    
    return polarity, -polarity, polarity, subjectivity, len(nltk.sent_tokenize(text)), complex_word_percentage, fog_index, len(words) / len(nltk.sent_tokenize(text)), complex_word_percentage / 100 * num_words, num_words, syllables_per_word, personal_pronoun_count, avg_word_length

# Apply the modified function to the DataFrame
df[['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']] = df['data'].apply(lambda x: pd.Series(calculate_scores(x)))

# Save the updated DataFrame to a new CSV file
#df.to_csv('updated_dataset.csv', index=False)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\px\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
df.drop(['data'],inplace = True , axis =1 )

In [25]:
df.to_excel('C:/Users/px/Desktop/Assignment_Final.xlsx', index=False)
